# Solution for Puzzle x

## Imports

In [19]:
import pandas as pd
from tqdm import tqdm

## Input Data

In [ ]:
input = pd.read_csv("input/day_1.csv", sep=",", header=None)
print(f"Number of ranges: {len(input)}")
input.head(5)

## Part One

In [ ]:
def parse_shape(lines):
    shape_coords = set()
    for r, line in enumerate(lines):
        for c, char in enumerate(line):
            if char == '#':
                shape_coords.add((r, c))
    if not shape_coords: return frozenset()
    min_r = min(r for r, c in shape_coords)
    min_c = min(c for r, c in shape_coords)
    normalized_shape = frozenset((r - min_r, c - min_c) for r, c in shape_coords)
    return normalized_shape

def get_shape_bounds(shape):
    if not shape: return 0, 0
    max_r = max(r for r, c in shape)
    max_c = max(c for r, c in shape)
    return max_r, max_c

def rotate(shape):
    """Rotate present shape."""
    if not shape: return frozenset()
    max_r, _ = get_shape_bounds(shape)
    new_coords = set()
    for r, c in shape:
        new_coords.add((c, max_r - r))
    min_r = min(r for r, c in new_coords)
    min_c = min(c for r, c in new_coords)
    return frozenset((r - min_r, c - min_c) for r, c in new_coords)

def flip(shape):
    """Flip present shape."""
    if not shape: return frozenset()
    _, max_c = get_shape_bounds(shape)
    new_coords = set()
    for r, c in shape:
        new_coords.add((r, max_c - c))
    min_c = min(c for r, c in new_coords)
    return frozenset((r, c - min_c) for r, c in new_coords)

def generate_orientations(base_shape):
    "Generate orientations for present."
    orientations = set()
    current_shape = base_shape
    for _ in range(4):
        orientations.add(current_shape)
        orientations.add(flip(current_shape))
        current_shape = rotate(current_shape)
    return list(orientations)

def place_shape(grid, shape, r_start, c_start, present_id):
    """Place present in grid."""
    H, W = len(grid), len(grid[0])
    placement_coords = []
    
    # 1. Check bounds and conflicts
    for r_offset, c_offset in shape:
        r, c = r_start + r_offset, c_start + c_offset
        if not (0 <= r < H and 0 <= c < W): return False 
        if grid[r][c] != 0: return False
        placement_coords.append((r, c))
    
    # 2. Place the shape
    for r, c in placement_coords:
        grid[r][c] = present_id
        
    return True

def remove_shape(grid, shape, r_start, c_start):
    """Corrected version"""
    for r_offset, c_offset in shape:
        r, c = r_start + r_offset, c_start + c_offset 
        grid[r][c] = 0

def solve_tiling(grid, H, W, required_presents, present_orientations):
    """
    Simplified recursive backtracking function that iterates over all possible 
    top-left placement points.
    """
    
    if not required_presents:
        return True
    
    shape_index = required_presents[0]
    current_present_id = shape_index + 1
    
    # The dimensions of the shape's current orientation's bounding box
    
    # Iterate through all orientations of the current shape
    for shape in present_orientations[shape_index]:
        max_r_offset, max_c_offset = get_shape_bounds(shape)
        
        # Iterate over all possible top-left placement points (r_start, c_start)
        # We only need to check points where the shape's bottom-right corner 
        # is still within the grid.
        for r_start in range(H - max_r_offset):
            for c_start in range(W - max_c_offset):
                
                if place_shape(grid, shape, r_start, c_start, current_present_id):
                    
                    # Placement successful, recurse with the rest
                    if solve_tiling(grid, H, W, required_presents[1:], present_orientations):
                        return True # Found a complete solution
                    
                    # Backtrack: Remove the shape
                    remove_shape(grid, shape, r_start, c_start)

    return False # Failed to place the current present


def fit_presents(input_content):
    """Main function to parse the input and solve the puzzle."""
    
    parts = input_content.strip().split('\n')
    split_idx = -1
    for i, line in enumerate(parts):
        line = line.strip()
        if 'x' in line and ':' in line and line.split(':')[0].strip().split('x')[0].isdigit():
            split_idx = i
            break
            
    if split_idx == -1: return 0

    shape_section = '\n'.join(parts[:split_idx])
    region_section = '\n'.join(parts[split_idx:])

    # Parse Shapes
    shape_definitions = shape_section.strip().split('\n\n')
    raw_shapes = {}
    
    for block in shape_definitions:
        lines = [line.strip() for line in block.strip().split('\n') if line.strip()]
        if not lines: continue
        
        if ':' in lines[0] and lines[0].split(':')[0].strip().isdigit():
            current_index = int(lines[0].split(':')[0].strip())
            shape_lines = lines[1:]
            raw_shapes[current_index] = shape_lines
            
    parsed_shapes = {}
    for index, lines in raw_shapes.items():
        parsed_shapes[index] = parse_shape(lines)

    all_present_orientations = {}
    for index, shape in parsed_shapes.items():
        all_present_orientations[index] = generate_orientations(shape)

    # Parse Regions
    solvable_regions = 0
    
    for line in tqdm(region_section.strip().split('\n')):
        line = line.strip()
        if not line: continue
        
        try:
            dims_str, reqs_str = line.split(':')
            W, H = map(int, dims_str.strip().split('x'))
            required_counts = list(map(int, reqs_str.strip().split()))
            
            required_presents = []
            for shape_index, count in enumerate(required_counts):
                required_presents.extend([shape_index] * count)
                
            total_present_area = sum(len(parsed_shapes.get(i, set())) for i in required_presents)
            if total_present_area > W * H: continue
            
            grid = [[0] * W for _ in range(H)]
            
            # Sort presents by area (largest first) to improve search speed
            required_presents.sort(key=lambda idx: len(parsed_shapes.get(idx, set())), reverse=True)
            
            if solve_tiling(grid, H, W, required_presents, all_present_orientations):
                solvable_regions += 1

        except ValueError:
            continue
            
    return solvable_regions

with open('input/day_12.csv', 'r') as f:
    input_content = f.read()

result = fit_presents(input_content)
print(f"{result} regions can fit all of the presents listed.")


100%|██████████| 1000/1000 [00:16<00:00, 61.69it/s]

440 regions can fit all of the presents listed.
